# Pytorch | 深入剖析torch.nn.Module方法及源码

torch.nn是一个专门为神经网络设计的模块化接口，包含卷积、池化、线性等计算，以及其他如loss等，可以将torch.nn中的每一个模块看做神经网络中的每一层。

torch.nn.Module是网络模型的一个基类，大部分自定义的子模型（卷积、池化甚至整个网络）是这个基类的子类。

## 一、`class torch.nn.Parameter`

功能：`torch.nn.Parameter`是继承至`torch.tensor`的子类，Parameter类型会自动被认为是module的可训练参数，即加入`.parameter()`迭代器中。

exp: `torch.nn.Parameter(torch.tensor[3.14159],requore_grad=True)`



## 二、`class torch.nn.Module`

### 2.1 构建模型

首先我们看看如何定义一个Module：

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

Module类包含48个方法，下面我们来看一下这些方法如何使用。

### 2.2 子模型操作

> register_module()

add_module方法的封装，用于将新的`name:module`键值对加入module中。

> add_module(name, module)

将子模块添加到当前模块。example：

In [4]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # 下面是两种等价方式
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.add_module("conv2", nn.Conv2d(1, 20, 5))

model = Model()
print(model)


Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
)


> 访问子模型

In [5]:
model.conv1

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))

> children()：

返回网络模型里的组成元素的迭代器。类似`modules()`方法。二者对比可参考：[https://blog.csdn.net/u013066730/article/details/94600978](https://blog.csdn.net/u013066730/article/details/94600978)

In [7]:
for i in model.children():
    print(i)

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))


> named_children()

返回直接子模块的迭代器，产生模块的名称以及模块本身。

In [10]:
for name, module in model.named_children():
    print(name,module)

conv1 Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
conv2 Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))


> `modules()`

返回当前模型所有模型的迭代器，重复的模型只被返回一次。与`children()`方法不同，`modules()`方法还会返回当前模型module

In [24]:
import torch

net = nn.Sequential(nn.Linear(2, 2), 
                    nn.Linear(2, 2),
                    nn.ReLU(),
                    nn.Sequential(nn.Linear(2, 2),
                                    nn.ReLU())
                    )
for module in net.modules():
    print(module)

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
  (2): ReLU()
  (3): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
  )
)
Linear(in_features=2, out_features=2, bias=True)
Linear(in_features=2, out_features=2, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
)
Linear(in_features=2, out_features=2, bias=True)
ReLU()


> named_modules

返回网络中所有模块的迭代器，产生模块的名称以及模块本身。

In [25]:
for name, module in model.named_modules():
    print(name, module)

 GaussianModel()


> get_submodule(target: str) -> 'Module'

从Module中获取子module，example：

### 2.3 模型参数（parameter）与缓冲区（buffer）

> register_parameter(self, name: str, param: Optional[Parameter])

用于在当前模块中添加一个parameter变量，其中参数param是一个Parameter类型（继承至tensor类型，nn.parameter.Parameter）。

Example:

In [21]:
import torch
import torch.nn as nn

class GaussianModel(nn.Module):

    def __init__(self):
        super(GaussianModel, self).__init__()

        self.register_parameter('mean', nn.Parameter(torch.zeros(1),
                                                     requires_grad=True))
        
        self.pdf = torch.distributions.Normal(self.state_dict()['mean'],
                                              torch.tensor([1.0]))
    def forward(self, x):
        return -self.pdf.log_prob(x)

model = GaussianModel()
for name, param in model.named_parameters():
    print(name,param.size())

mean torch.Size([1])


> parameters(recurse=True)

返回模型参数的迭代器

In [17]:
for param in model.parameters():
    print(type(param), param.size())

> named_parameters(prefix='', recurse=True)

返回模块参数的迭代器，产生参数的名称以及参数本身。

In [15]:
for name, param in net.named_parameters():
    print(name,param.size())

0.weight torch.Size([2, 2])
0.bias torch.Size([2])
1.weight torch.Size([2, 2])
1.bias torch.Size([2])
3.0.weight torch.Size([2, 2])
3.0.bias torch.Size([2])


> get_parameter(target: str)

根据参数名得到参数，exp：

In [29]:
net.get_parameter('1.weight')

Parameter containing:
tensor([[-0.1466, -0.1264],
        [ 0.2812,  0.1436]], requires_grad=True)

> buffers(recurse=True)

模型中需要保存下来的参数包括两种：

+ 一种是反向传播需要更新：parameter，可以通过parameter()返回
+ 一种是反向传播不需要更新的：buffer，可以通过buffer()返回

> named_buffers(prefix='', recurse=True)

返回module buffers' name的迭代器，example： 

In [20]:
for name, buf in net.named_buffers():
    print(buf.size())

> register_buffer(name: str, tensor: Optional[Tensor], persistent: bool = True)

在当前模块中添加一个buffer变量，例如，现在需要手写一个BatchNorm，那么其`running_mean`并不是一个parameter，这就需要用下述方式注册一个buffer：

```python
class BatchNorm(nn.Module):
    def __init__(self,..):
        self.register_buffer('running_mean',torch.zeros(num_features))
        self.register_buffer('running_variance',torch.ones(num_features))
```

> get_buffer(target: str)

根据buffer名得到buffer值，用法同get_parameter。

### 2.4 数据格式及转换

> float() / double() / half() / bfloat16() 

将所有的parameters和buffers转化为指定的数据类型。

> type(dst_type)

将所有的parameters和buffers转化为目标数据类型。

### 2.5 模型移动

> to_empty()

把模型parameter和buffers移动到指定device上（不保存其具体数值）。

> cpu() / cuda() / xpu(）

将模型的parameters和buffers移动到CPU/GPU

### 2.5 模型模式调整

> train(mode=True)
将该模块设置为train训练模式。默认值：True。

> eval()

将module设置为验证模式，会影响一些特定modules，如：Dropout，BatchNorm等

### 2.6 其他

> zero_grad(set_to_none=False)

将所有模型参数的梯度设置为零。set_to_none=True会让内存分配器来处理梯度，而不是主动将它们设置为0，这样会适度加速。

> forward(*input)

方法定义了神经网络每次调用时都需要执行的前向传播计算，所有的子类都必须要重写这个方法。

> apply(fn)

+ 递归地将函数应用于所有子模块。
+ apply方法可以用于任何submodule（通过.children()或者self.获取到的）
+ 常用来初始化模型参数（同torch.nn.init）

example:


In [30]:
import torch
@torch.no_grad()  # 不计算梯度，不反向传播
def init_weights(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.fill_(1.0)
        print(m.weight)
net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2),nn.ReLU(),nn.Sequential(nn.Linear(2, 2),nn.ReLU()))
net.apply(init_weights)

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
ReLU()
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
ReLU()
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
  (2): ReLU()
  (3): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
  )
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
  (2): ReLU()
  (3): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
  )
)

> load_state_dict(state_dict, strict=True)

将 state_dict 中的参数(parameters)和缓冲区(buffers)复制到此模块及其子模块中。如果 strict 为 True，则 state_dict 的键必须与该模块的 state_dict() 函数返回的键完全匹配。

> state_dict()

返回包含模块整个状态的字典。 包括参数和持久缓冲区（例如运行平均值）。键是对应的参数和缓冲区名称。不包括设置为 None 的参数和缓冲区。常用于保存模型参数。

保存模型例子：

```python
# Additional information
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)
```

加载模型例子：
```python
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()
```
更多详情参考：[https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html)

> _apply(fn)

+ 对所有的module、parameter、buffer都进行一个fn

Example：.cpu / .cuda()源码

```python
class Module:
    def cuda(self: T, device: Optional[Union[int, device]] = None) -> T:
        r"""Moves all model parameters and buffers to the GPU.

        This also makes associated parameters and buffers different objects. So
        it should be called before constructing optimizer if the module will
        live on GPU while being optimized.

        .. note::
            This method modifies the module in-place.

        Args:
            device (int, optional): if specified, all parameters will be
                copied to that device

        Returns:
            Module: self
        """
        return self._apply(lambda t: t.cuda(device))
```

## 三、hook方法

In [32]:
import timm
import torch

ImportError: cannot import name '_log_api_usage_once' from 'torchvision.utils' (/Users/liujiyao/miniforge3/envs/torchcpu/lib/python3.8/site-packages/torchvision/utils.py)